In [ ]:
!pip install git+https://github.com/bstriner/keras-adversarial.git

In [ ]:
!pip install Keras==2.1.1

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Reshape
from keras.regularizers import L1L2
from keras_adversarial import AdversarialModel,simple_gan,gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling

In [ ]:
(previsores_treinamento,_),(_,_) = mnist.load_data()
previsores_treinamento = previsores_treinamento.astype("float32")/255

In [ ]:
previsores_treinamento.shape

In [ ]:
gerador = Sequential()
gerador.add(Dense(units=500, input_dim=100,activation= 'relu', kernel_regularizer=L1L2(1e-5,1e-5)))
gerador.add(Dense(units=500, input_dim=100,activation= 'relu', kernel_regularizer=L1L2(1e-5,1e-5)))
gerador.add(Dense(units=784,activation= 'relu', kernel_regularizer=L1L2(1e-5,1e-5))) #28x 28 = 784
gerador.add(Reshape((28,28)))

In [ ]:
discriminador = Sequential()
discriminador.add(InputLayer(input_shape=(28,28)))
discriminador.add(Flatten())
discriminador.add(Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5,1e-5)))
discriminador.add(Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5,1e-5)))
discriminador.add(Dense(units=1, activation='sigmoid', kernel_regularizer=L1L2(1e-5,1e-5)))

In [10]:
gan = simple_gan(gerador,discriminador, normal_latent_sampling((100,)))

In [11]:
modelo = AdversarialModel(base_model = gan, player_params = [gerador.trainable_weights,discriminador.trainable_weights])

In [12]:
modelo.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(), player_optimizers=['adam', 'adam'], loss = 'binary_crossentropy')

In [ ]:
modelo.fit(x=previsores_treinamento,y=gan_targets(60000), epochs= 1000, batch_size = 256)

In [ ]:
amostras = np.random.normal(size=(10,100))
previsao = gerador.predict(amostras)
for i in range(previsao.shape[0]):
    plt.imshow(previsao[i,:], cmap='gray')
    plt.show()